## **Plotly Performance - Investigation**

### Instructions:
- install plotly-resample: `pip install plotly-resampler` (https://github.com/predict-idlab/plotly-resampler)
- execute this notebook: once on your local environment AND once in your online environment (e.g. jupyterlab)
- Have a look at the execution times of Test 1 - 4 and
fill your infos into this first table: https://docs.google.com/document/d/1ruplgpMVA-rjwkSSvZvYMfnCNkBiTAX6A8i5impuKZE/edit?usp=sharing
- additionally, execute the `temp_plotly_performance.py` and fill your info
in the second table of the file linked above

Thank you!

#####  Import libraries and extract data from github


In [1]:

import pandas as pd
import plotly

from neuralprophet import set_log_level
from plotly_resampler import register_plotly_resampler
# from plotly_resampler.figure_resampler import display
# from IPython.display import display
import plotly.io as pio
import time
import psutil

set_log_level("ERROR")

#####  Get number of CPUs in the system

In [48]:
import os

# Get the number of CPUs in the system
num_cores = os.cpu_count()

# Print the number of used cores
print(f'Number of used cores: {num_cores}')

##### Get plotly default renderer

In [47]:
print("plotly default renderer:", pio.renderers.default)


##### Prepare test data

In [4]:
data_location = "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/"
file = "energy/SF_hospital_load.csv"

In [49]:
data_df = pd.read_csv(data_location + file)
data_df.shape

In [50]:
# Create df with 2x number of rows
rng_2x = pd.date_range(start='2015-01-01 01:00:00', freq='H', periods=2*8760)
y_2x = pd.concat([data_df['y'], data_df['y']], ignore_index=True)
data_df_2x = pd.DataFrame({'ds': rng_2x, 'y': y_2x})
data_df_2x.tail(5)

In [51]:
# Create df with 10x number of rows*
rng_10x = pd.date_range(start='2015-01-01 01:00:00', freq='H', periods=10*8760)
y_10x = pd.concat([data_df_2x['y'], data_df_2x['y'], data_df_2x['y'], data_df_2x['y'], data_df_2x['y']], ignore_index=True)
data_df_10x = pd.DataFrame({'ds': rng_10x, 'y': y_10x})
data_df_10x.shape

##### Define, fit, and predict models

In [8]:
quantile_lo, quantile_hi = 0.05, 0.95
quantiles = [quantile_lo, quantile_hi]

In [9]:
from neuralprophet import NeuralProphet

m = NeuralProphet(
    epochs=1,
    batch_size=128,
    learning_rate=1.0,
    n_forecasts=7,
    n_lags=14,
    quantiles=quantiles
)

m2x = NeuralProphet(
    epochs=1,
    batch_size=128,
    learning_rate=1.0,
    n_forecasts=7,
    n_lags=14,
    quantiles=quantiles
)

m10x = NeuralProphet(
    epochs=1,
    batch_size=128,
    learning_rate=1.0,
    n_forecasts=7,
    n_lags=14,
    quantiles=quantiles
)

In [53]:
metrics = m.fit(data_df, freq="H")

In [54]:
metrics2x = m2x.fit(data_df_2x, freq="H")

In [52]:
metrics10x = m10x.fit(data_df_10x, freq="H")

In [57]:
forecast = m.predict(data_df)

In [56]:
forecast2x= m2x.predict(data_df_2x)

In [55]:
forecast10x = m10x.predict(data_df_10x)

### Plotting

In [16]:
# register plotly-resampler in 'auto' mode
register_plotly_resampler(mode='auto')

### Test 1

In [58]:
# get the start time
st = time.time()

fig1 = m.highlight_nth_step_ahead_of_each_forecast(1).plot(
    forecast, plotting_backend='plotly'
).show()

# get the end time
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
print('RAM memory % used:', psutil.virtual_memory()[2])
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

### Test 2

In [59]:
# get the start time
st = time.time()

fig2 = m10x.highlight_nth_step_ahead_of_each_forecast(1).plot(
    forecast10x, plotting_backend='plotly'
).show()

# get the end time
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
print('RAM memory % used:', psutil.virtual_memory()[2])
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

### Test 3

In [60]:
# get the start time
st = time.time()

fig3 = m.highlight_nth_step_ahead_of_each_forecast(1).plot(
    forecast, plotting_backend='matplotlib'
).show()

# get the end time
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
print('RAM memory % used:', psutil.virtual_memory()[2])
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

### Test 4

In [62]:
# get the start time
st = time.time()

fig4 = m10x.highlight_nth_step_ahead_of_each_forecast(1).plot(
    forecast10x, plotting_backend='matplotlib'
).show()

# get the end time
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
print('RAM memory % used:', psutil.virtual_memory()[2])
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)